In [5]:
# 03_log_big_opening
# join a log with BID opening event extracted from texts

In [6]:
import pandas as pd
import os
import csv

In [7]:
# GLOBALS
log_dir = "log_filtrato_DISCO" # <-- INPUT: log dir to save the files
file_bid = "event_log_IT_BID-OPENING.csv" # <-- INPUT: log with BID-OPENING event
file_log = "event_log_IT.csv" # <-- INPUT: log to join with

In [8]:
# Open the main log
dic_t = {'Case_ID':object}
path_data = os.path.join(log_dir, file_log)
df_log = pd.read_csv(path_data, sep = ";", dtype=dic_t, low_memory=False)
df_log = df_log.rename({'Case_ID':'case_id'}, axis = 1)

In [9]:
df_log.columns

Index(['case_id', 'Activity', 'Complete Timestamp', 'Variant', 'Variant index',
       'type', 'amount', 'electronic', 'framework_agr', 'nuts', 'cpv_main',
       'country'],
      dtype='object')

In [10]:
# Eventual clean
list_col_del = ['Variant', 'Variant index']
for col in df_log.columns:
    if col in list_col_del:
        df_log = df_log.drop(columns = [col], axis=1)

In [11]:
# Show DF
df_log

,case_id,Activity,Complete Timestamp,type,amount,electronic,framework_agr,nuts,cpv_main,country
0,20162872,PUBLICATION,2016-01-04 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
1,20162872,PARTECIPATION,2016-02-23 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
2,20162872,AWARD,2016-03-18 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
3,20162872,CONTRACT-START,2016-04-01 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
4,20162872,CONTRACT-END,2021-03-31 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
...,...,...,...,...,...,...,...,...,...,...
5151,2022423809,PUBLICATION,2022-07-29 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT
5152,2022423809,PARTECIPATION,2022-08-23 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT
5153,2022423809,AWARD,2022-09-09 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT
5154,2022423809,CONTRACT-START,2022-09-10 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT


In [12]:
len(df_log['case_id'].unique())

1063

In [13]:
# Get only one case distinct
df_log_caseid = df_log.drop_duplicates(subset=['case_id'])
df_log_caseid.head()

,case_id,Activity,Complete Timestamp,type,amount,electronic,framework_agr,nuts,cpv_main,country
0,20162872,PUBLICATION,2016-01-04 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
5,20164078,PUBLICATION,2016-01-05 00:00:00.000,S,NaN,N,N,NaN,92.0,IT
10,20169964,PUBLICATION,2016-01-11 00:00:00.000,S,NaN,N,N,NaN,601.0,IT
15,201615326,PUBLICATION,2016-01-14 00:00:00.000,S,NaN,N,N,NaN,66.0,IT
20,201617147,PUBLICATION,2016-01-14 00:00:00.000,S,4006227.27,N,N,NaN,85.0,IT


In [14]:
df_log_caseid.shape

(1063, 10)

In [15]:
# Remove Activity, Complete Timestamp to have only static attributes
df_log_caseid = df_log_caseid.drop(columns = ['Activity', 'Complete Timestamp'], axis = 1)

In [16]:
df_log_caseid.shape

(1063, 8)

In [17]:
# Open the BID-OPENING log
dic_t = {'case_id':object}
path_data = os.path.join(log_dir, file_bid)
df_log_bid = pd.read_csv(path_data, sep = ";", dtype=dic_t, low_memory=False)
if 'file_source' in df_log_bid.columns:
    df_log_bid = df_log_bid.drop(columns = ['file_source'], axis=1)
# Rename 
df_log_bid = df_log_bid.rename({'event':'Activity'}, axis = 1)
df_log_bid = df_log_bid.rename({'new_date':'Complete Timestamp'}, axis = 1)

In [18]:
df_log_bid

,case_id,Activity,Complete Timestamp
0,20162872,BID-OPENING,2016-02-25
1,20169964,BID-OPENING,2016-02-25
2,201615326,BID-OPENING,2016-03-11
3,201617147,BID-OPENING,2016-03-04
4,201635271,BID-OPENING,2016-03-10
...,...,...,...
264,2018562401,BID-OPENING,2019-02-19
265,2018566542,BID-OPENING,2019-01-29
266,2018571813,BID-OPENING,2019-01-21
267,2018571840,BID-OPENING,2019-03-01


In [19]:
df_log_bid['case_id'].nunique()

269

In [20]:
# add the attributes
df_log_bid_att = pd.merge(left = df_log_bid, right = df_log_caseid, on = 'case_id', how = 'inner')

In [21]:
df_log_bid_att.head()

,case_id,Activity,Complete Timestamp,type,amount,electronic,framework_agr,nuts,cpv_main,country
0,20162872,BID-OPENING,2016-02-25,S,1473151.90,N,N,NaN,90.0,IT
1,20169964,BID-OPENING,2016-02-25,S,NaN,N,N,NaN,601.0,IT
2,201615326,BID-OPENING,2016-03-11,S,NaN,N,N,NaN,66.0,IT
3,201617147,BID-OPENING,2016-03-04,S,4006227.27,N,N,NaN,85.0,IT
4,201635271,BID-OPENING,2016-03-10,S,1055000.00,NaN,N,NaN,50.0,IT


In [22]:
df_log_bid_att.shape

(269, 10)

In [23]:
# Concate the df_bid with static columns to df_log

In [24]:
df_concat = pd.concat([df_log, df_log_bid_att])

In [25]:
df_concat

,case_id,Activity,Complete Timestamp,type,amount,electronic,framework_agr,nuts,cpv_main,country
0,20162872,PUBLICATION,2016-01-04 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
1,20162872,PARTECIPATION,2016-02-23 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
2,20162872,AWARD,2016-03-18 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
3,20162872,CONTRACT-START,2016-04-01 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
4,20162872,CONTRACT-END,2021-03-31 00:00:00.000,S,1473151.90,N,N,NaN,90.0,IT
...,...,...,...,...,...,...,...,...,...,...
264,2018562401,BID-OPENING,2019-02-19,S,135885.40,NaN,N,NaN,NaN,IT
265,2018566542,BID-OPENING,2019-01-29,S,230940.00,N,N,NaN,NaN,IT
266,2018571813,BID-OPENING,2019-01-21,S,729902.88,N,N,NaN,NaN,IT
267,2018571840,BID-OPENING,2019-03-01,S,450560.00,NaN,N,NaN,NaN,IT


In [26]:
# Order
df_concat = df_concat.sort_values(by=['case_id', 'Complete Timestamp'], ascending = True)

In [27]:
df_concat

,case_id,Activity,Complete Timestamp,type,amount,electronic,framework_agr,nuts,cpv_main,country
137,2016102606,PUBLICATION,2016-03-24 00:00:00.000,S,2029500.00,NaN,N,IT,85.0,IT
138,2016102606,PARTECIPATION,2016-04-18 00:00:00.000,S,2029500.00,NaN,N,IT,85.0,IT
139,2016102606,CONTRACT-START,2016-09-01 00:00:00.000,S,2029500.00,NaN,N,IT,85.0,IT
140,2016102606,AWARD,2016-09-21 00:00:00.000,S,2029500.00,NaN,N,IT,85.0,IT
141,2016102606,CONTRACT-END,2021-08-22 00:00:00.000,S,2029500.00,NaN,N,IT,85.0,IT
...,...,...,...,...,...,...,...,...,...,...
5151,2022423809,PUBLICATION,2022-07-29 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT
5152,2022423809,PARTECIPATION,2022-08-23 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT
5153,2022423809,AWARD,2022-09-09 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT
5154,2022423809,CONTRACT-START,2022-09-10 00:00:00.000,S,230613.23,N,N,ITC48,NaN,IT


In [28]:
# Save
file_out = "event_log_IT_join_BID-OPENING.csv"
path_data = os.path.join(log_dir, file_out)
df_concat.to_csv(path_data, sep = ";", index = False, quoting = csv.QUOTE_ALL)